The almanac (your puzzle input) lists all of the seeds that need to be planted. It also lists what type of soil to use with each kind of seed, what type of fertilizer to use with each kind of soil, what type of water to use with each kind of fertilizer, and so on. Every type of seed, soil, fertilizer and so on is identified with a number, but numbers are reused by each category - that is, soil 123 and fertilizer 123 aren't necessarily related to each other.

For example:
```
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
```

The almanac starts by listing which seeds need to be planted: seeds 79, 14, 55, and 13.

The rest of the almanac contains a list of maps which describe how to convert numbers from a source category into numbers in a destination category. That is, the section that starts with seed-to-soil map: describes how to convert a seed number (the source) to a soil number (the destination). This lets the gardener and his team know which soil to use with which seeds, which water to use with which fertilizer, and so on.

Rather than list every source number and its corresponding destination number one by one, the maps describe entire ranges of numbers that can be converted. Each line within a map contains three numbers: the destination range start, the source range start, and the range length.

Consider again the example seed-to-soil map:
```
50 98 2
52 50 48
```
The first line has a destination range start of 50, a source range start of 98, and a range length of 2. This line means that the source range starts at 98 and contains two values: 98 and 99. The destination range is the same length, but it starts at 50, so its two values are 50 and 51. With this information, you know that seed number 98 corresponds to soil number 50 and that seed number 99 corresponds to soil number 51.

The second line means that the source range starts at 50 and contains 48 values: 50, 51, ..., 96, 97. This corresponds to a destination range starting at 52 and also containing 48 values: 52, 53, ..., 98, 99. So, seed number 53 corresponds to soil number 55.

Any source numbers that aren't mapped correspond to the same destination number. So, seed number 10 corresponds to soil number 10.

So, the entire list of seed numbers and their corresponding soil numbers looks like this:
```
seed  soil
0     0
1     1
...   ...
48    48
49    49
50    52
51    53
...   ...
96    98
97    99
98    50
99    51
```
With this map, you can look up the soil number required for each initial seed number:

- Seed number 79 corresponds to soil number 81.
- Seed number 14 corresponds to soil number 14.
- Seed number 55 corresponds to soil number 57.
- Seed number 13 corresponds to soil number 13.

The gardener and his team want to get started as soon as possible, so they'd like to know the closest location that needs a seed. Using these maps, find the lowest location number that corresponds to any of the initial seeds. To do this, you'll need to convert each seed number through other categories until you can find its corresponding location number. In this example, the corresponding types are:

- Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
- Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
- Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
- Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.

So, the lowest location number in this example is 35.

What is the lowest location number that corresponds to any of the initial seed numbers?

In [1]:
input = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [ ]:
# with open('input.txt', 'r') as f:
#     input = f.read()

In [2]:
class Mapping:
    def __init__(self, dest_rs, src_rs, r_length):
        self.dest_rs = dest_rs
        self.src_rs = src_rs
        self.r_length = r_length
        self.range = (src_rs, src_rs + r_length - 1)

    def __repr__(self):
        return "Mapping({}, {}, {})".format(self.dest_rs, self.src_rs, self.r_length)
    
    def get_one_way_mapping(self, x):
        '''
        returns dest_rs + (x - src_rs) if x is in range [src_rs, src_rs + r_length)
        else returns x
        '''
        if x >= self.src_rs and x < (self.src_rs + self.r_length):
            return self.dest_rs + (x - self.src_rs)
        else:
            return x
        
    def get_reverse_mapping(self, x):
        '''
        returns src_rs + (x - dest_rs) if x is in range [dest_rs, dest_rs + r_length)
        else returns x
        '''
        if x >= self.dest_rs and x < (self.dest_rs + self.r_length):
            return self.src_rs + (x - self.dest_rs)
        else:
            return x

In [3]:
chunks = input.split("\n\n")

# Seeds is the first chunk.
seeds = chunks[0][chunks[0].index(":") + 2:].split(" ")

# convert seeds to ints.
for i in range(len(seeds)):
    seeds[i] = int(seeds[i])

chunks = chunks[1:]

maps = {}
for chunk in chunks:
    lines = chunk.split("\n")
    # Line 1 formatted like ("{x}-to-{y} map:") - (x, y) is the key.
    key = tuple(lines[0].split(" ")[0].split("-")[0::2])
    lines = lines[1:]

    maps[key] = []
    # The rest of the lines are the mappings.
    for line in lines:
        # if line is empty, skip it.
        if line == "":
            continue

        # Add a new Mapping based on the line.
        maps[key].append(Mapping(*[int(s) for s in line.split(" ")]))


In [4]:
locations = []

# For each seed, apply the mappings in order.
for seed in seeds:
    # Start with the seed.
    x = seed
    # Apply the mappings in order.
    for key in maps:
        for mapping in maps[key]:
            temp = x
            x = mapping.get_one_way_mapping(x)

            # If the mapping changed x, don't apply any more mappings.
            if temp != x:
                break
    # Add the final x to the locations.
    locations.append(x)

print(sorted(locations))

[35, 43, 82, 86]


Everyone will starve if you only plant such a small number of seeds. Re-reading the almanac, it looks like the seeds: line actually describes ranges of seed numbers.

The values on the initial seeds: line come in pairs. Within each pair, the first value is the start of the range and the second value is the length of the range. So, in the first line of the example above:
`seeds: 79 14 55 13`
This line describes two ranges of seed numbers to be planted in the garden. The first range starts with seed number 79 and contains 14 values: 79, 80, ..., 91, 92. The second range starts with seed number 55 and contains 13 values: 55, 56, ..., 66, 67.

This line describes two ranges of seed numbers to be planted in the garden. The first range starts with seed number 79 and contains 14 values: 79, 80, ..., 91, 92. The second range starts with seed number 55 and contains 13 values: 55, 56, ..., 66, 67.

Now, rather than considering four seed numbers, you need to consider a total of 27 seed numbers.

In the above example, the lowest location number can be obtained from seed number 82, which corresponds to soil 84, fertilizer 84, water 84, light 77, temperature 45, humidity 46, and location 46. So, the lowest location number is 46.

In [ ]:
# A seed range is [seeds[i](even), seeds[i] + seeds[i+1]).
seed_ranges = [(seeds[i], seeds[i] + seeds[i+1] - 1) for i in range(0, len(seeds), 2)]

print(seed_ranges)

# Check every location, starting at 0, to see if it is in a seed range.
result = 0
while True:
    # apply the mappings in reverse order.
    x = result
    for key in maps:
        for mapping in maps[key]:
            temp = x
            x = mapping.get_reverse_mapping(x)

            print(x)

            # If the mapping changed x, don't apply any more mappings.
            if temp != x:
                break

    # Check if x is in a seed range.
    for seed_range in seed_ranges:
        if x >= seed_range[0] and x <= seed_range[1]:
            # If it is, we're done.
            print(result)
            break

    # If we're done, break out of the loop.
    if x >= seed_range[0] and x <= seed_range[1]:
        break

    # If not, increment result and try again.
    result += 1